In [1]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate
# from common.models.place import Event
from langchain_core.language_models.chat_models import BaseChatModel
from loguru import logger
import json
from pydantic import BaseModel
from collections import Counter
from typing import Tuple, Dict, List
from langchain_ollama import ChatOllama
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field, ConfigDict


In [2]:
load_dotenv()

model = ChatOllama(
	base_url=os.environ.get("OLLAMA_HOST"),
	model="llama3.1:8b",
	num_gpu=-1,
	validate_model_on_init=True,
	temperature=0.3
)

In [3]:

class Response(BaseModel):
	'''Model output for a single-option classification task.'''

	model_config = ConfigDict(str_strip_whitespace=True, extra="forbid")
	
	thinking: str = Field(
		...,
		description=(
			"Brief justification explaining why the selected option best matches "
			"the input text. This field is for interpretability only."
		),
		examples=[
			"The text sets up the hero's normal life and environment before any conflict occurs.",
			"The antagonist performs harmful acts that create obstacles for the hero.",
			"The hero is forced out of a safe place due to circumstances beyond their control.",
			"There is a direct confrontation involving physical combat."
		],
		min_length=1 # No permite strings vacíos
	)

	response: int = Field(
		...,
		description=(
			"Zero-based index of the selected option from the provided list. "
			"The value must correspond to exactly one available option."
		),
		examples=[0, 2, 5],
		ge=0  # mayor o igual que 0
		)


In [4]:

system_prompt = SystemMessagePromptTemplate.from_template(template="""You are an expert narrative analysis assistant. Your task is to classify a text fragment by selecting the most specific narrative event from a closed list of options.

Instructions:
- Select exactly ONE option from the list.
- Each option is identified by its index number (0, 1, 2, ...).
- Return ONLY the index number as an integer in "response".
- Do NOT invent options or return natural language in "response".
- If multiple options are applicable, select the one that is the most detailed and specific.
- Abstract or general options should only be chosen if no specific option applies.
Available options:
\"\"\"{options}\"\"\"

{previous_thought}
Output format:
{{
  "thinking": "Brief justification based on the text",
  "response": int
}}
""")

human_prompt = HumanMessagePromptTemplate.from_template(
	"""Text to classify:
\"\"\"{event}\"\"\""""
)

event_prompt = ChatPromptTemplate.from_messages([
	system_prompt,
	human_prompt,
])

In [5]:

def build_options_prompt(
	node: Dict,
	self_name: str = None
) -> str:
	options = []
	
	for child_id, info in node.get("children", {}).items():
		options.append((child_id, info["description"]))

	if self_name:
		options.append((self_name, node["description"]))

	lines = [
		f"{idx}. {node_id}: {description}" 
		for idx, (node_id, description) in enumerate(options)
	]

	return "\n".join(lines), options

def build_options_prompt_by_list(
	options: list
) -> str:
	lines = [
		f"{idx}. {node_id}: {description}" 
		for idx, (node_id, description) in enumerate(options)
	]
	return "\n".join(lines)

In [6]:
with open("data/event_with_descriptions.json", "r") as f:
	taxonomy_tree = json.load(f)

In [7]:
options_prompt = []
for node, info in taxonomy_tree["children"].items():
	options_prompt.append(f"- {node}: {info['description']}")
options_str = "\n".join(options_prompt)

In [8]:
print(options_str)

- move: Main action of the story, including setup, conflict, preparation, counteractions, and external help that drives the narrative.
- resolution: Closure of the story with conflict resolution, hero's victory, recognition, punishment of the villain, transformations, and symbolic closure such as weddings or thrones.


In [9]:
options_prompt = []
for node, info in taxonomy_tree["children"]["move"]["children"].items():
	options_prompt.append(f"- {node}: {info['description']}")
options_str = "\n".join(options_prompt)

In [10]:
print(options_str)

- setup: Initial context and situation of the hero, showing their state, environment, and relationships before any conflict.
- conflict: Problems and obstacles faced by the hero, including interdictions, lacks, struggles, marks received, connective incidents, and villain advantages.
- preparation: Planning and acquiring resources to face challenges, including temporary absence, violation of interdictions, obtaining objects and allies, guidance from mentors, strategies, and contact with the enemy.
- beginning_of_counteraction: Start of the hero's active response to conflicts and obstacles.
- helper_move: Intervention of allies or external forces to support the hero, including obtaining objects, resolving lacks, and rescues.
- false_hero_make_unfounded_claim: Appearance of a false hero who claims undeserved credit.
- attempt_at_reconnaissance: Attempts to explore or gather information about the enemy to plan action.


In [11]:
options,options_list =build_options_prompt(taxonomy_tree)

In [12]:
options

"0. move: Main action of the story, including setup, conflict, preparation, counteractions, and external help that drives the narrative.\n1. resolution: Closure of the story with conflict resolution, hero's victory, recognition, punishment of the villain, transformations, and symbolic closure such as weddings or thrones."

In [13]:
def extract_event(model: BaseChatModel, folktale_event: str, options:str, previous_thought:str = "" ):
	event_chain = event_prompt | model.with_structured_output(Response)
	response = event_chain.invoke({
		"options": options,
		"event": folktale_event,
		"previous_thought": previous_thought
	})
	return response.response, response.thinking

---


In [14]:
texto= "The suitor swore before the court that he would marry the princess and bind his fate to the royal family. He allowed the wedding to be proclaimed and accepted the gifts and honors of a future husband. Yet in secret he intended to flee the kingdom once he had secured the dowry, having never meant to fulfill the promised marriage."
id,t=extract_event(model,texto,options)

In [15]:
options_list[id]

('move',
 'Main action of the story, including setup, conflict, preparation, counteractions, and external help that drives the narrative.')

In [22]:

def hierarchical_event_classification_with_desc(
	model: BaseChatModel,
	folktale_event: str,
	taxonomy_tree: Dict,
	n_rounds: int = 3,
	verbose: bool = False
) -> Tuple[str, str]:
	"""
	Clasifica un evento usando una taxonomía jerárquica con descripciones.

	Args:
		model: Instancia de BaseChatModel.
		folktale_event: Texto del evento a clasificar.
		taxonomy_tree: Diccionario de taxonomía con estructura {node: {"description": ..., "children": {...}}}.
		n_rounds: Número de veces a preguntar al LLM por cada nivel.

	Returns:
		Tuple[str, str]: (evento final elegido, justificación final)
	"""

	current_nodes = taxonomy_tree["children"]
	previous_event = None
	final_thinking = []
	options_str,options_list = build_options_prompt(taxonomy_tree)
	level = 0

	if verbose:
		print("=== Inicio de clasificación jerárquica ===")
		print(f"Evento a clasificar: {folktale_event}")

	while current_nodes:
		votes = []
		thoughts = []
		final_thinking_str = "Previous decision or reasoning to consider:\n" + "\n".join(final_thinking)

		if verbose:
			print(f"\n--- Nivel {level} ---")
			print("Opciones disponibles:")
			print(options_str)

		# Preguntar al LLM n_rounds veces
		for i in range(n_rounds):
			event, thinking = extract_event(
				model=model,
				folktale_event=folktale_event,
				options=options_str,
				previous_thought=final_thinking_str
			)
			votes.append(event)
			thoughts.append(thinking)

			if verbose:
				print(f"\nLlamada al modelo ({i + 1}/{n_rounds})")
				print(f"  Evento propuesto: {options_list[event]}")
				print(f"  Justificación: {thinking}")

		if verbose: print("\n---\n")

		# Voto por mayoría
		vote_count = Counter(votes)
		max_freq = max(vote_count.values())
		most_frequent = [v for v, c in vote_count.items() if c == max_freq]

		winning_event = most_frequent[0]
		# winning_event, _ = vote_count.most_common(1)[0]

		if len(most_frequent)>1:
			selected = [options_list[i] for i in most_frequent]
			selected_str = build_options_prompt_by_list(selected)
			event, thinking = extract_event(
				model=model,
				folktale_event=folktale_event,
				options=selected_str,
				previous_thought=final_thinking_str
			)
			winning_event = most_frequent[event]

			if verbose:
				print(f" Empate:")
				print(selected_str)
				print(f"  Evento propuesto: {selected[event]}")
				print(f"  Justificación: {thinking}")

		final_thinking.extend(
			thoughts[i] for i, v in enumerate(votes) if v == winning_event
		)
		
		winning_event = options_list[winning_event][0]

		if verbose:print(f"  Evento propuesto: {winning_event}")

		# Si el evento se repite o no tiene hijos
		if winning_event == previous_event:
			if verbose:
				print("Evento repetido. Finalizando clasificación.")
			return winning_event, final_thinking

		if not current_nodes[winning_event]["children"]:
			if verbose:
				print("El evento ganador no tiene hijos. Finalizando clasificación.")
			return winning_event, final_thinking
		

		options_str,options_list = build_options_prompt(current_nodes[winning_event],winning_event)

		current_nodes = current_nodes[winning_event]["children"]
		previous_event = winning_event

		if verbose:print(f"Descendiendo a los hijos de: {winning_event}")
		level+=1

	if verbose:
		print("\n=== Fin de clasificación ===")
		print(f"  Evento propuesto: {previous_event}")
		print(f"  Justificación: {final_thinking}")

	return previous_event, final_thinking



In [23]:
texto = "The suitor publicly pledged to marry the princess, participated in the wedding preparations, and accepted all gifts and honors of a future husband. However, he secretly planned to abandon the kingdom immediately after receiving the dowry, having never intended to honor the marriage contract. His actions constitute a deliberate deception regarding matrimony, aimed at personal gain while misleading the royal family and the princess."
final_event, final_thinking = hierarchical_event_classification_with_desc(
	model=model,
	folktale_event=texto,
	taxonomy_tree=taxonomy_tree,
	n_rounds=5,
	verbose = True
)

# print("Evento final:", final_event)
# print("Justificación:", final_thinking)

=== Inicio de clasificación jerárquica ===
Evento a clasificar: The suitor publicly pledged to marry the princess, participated in the wedding preparations, and accepted all gifts and honors of a future husband. However, he secretly planned to abandon the kingdom immediately after receiving the dowry, having never intended to honor the marriage contract. His actions constitute a deliberate deception regarding matrimony, aimed at personal gain while misleading the royal family and the princess.

--- Nivel 0 ---
Opciones disponibles:
0. move: Main action of the story, including setup, conflict, preparation, counteractions, and external help that drives the narrative.
1. resolution: Closure of the story with conflict resolution, hero's victory, recognition, punishment of the villain, transformations, and symbolic closure such as weddings or thrones.

Llamada al modelo (1/5)
  Evento propuesto: ('move', 'Main action of the story, including setup, conflict, preparation, counteractions, and 

In [24]:
file= "data/event_test.json"

with open(file, "r", encoding="utf-8") as f:
	event_data = json.load(f)
for sample in event_data["samples"]:

	event, thinking = hierarchical_event_classification_with_desc(
		model=model,
		folktale_event=sample["text"],
		taxonomy_tree=taxonomy_tree,
		n_rounds=5,
		verbose = False
	)
	print("ID:", sample["id"])
	print("Type:", sample["type"])
	print("Text:", sample["text"])
	print("---")
	print("Thinking:", thinking)
	print("Response:", event)
	print("---")
	# if sample["type"] != event:
	# 	print("ID:", sample["id"])
	# 	print("Type:", sample["type"])
	# 	print("Text:", sample["text"])
	# 	print("---")
	# 	print("Thinking:", thinking)
	# 	print("Response:", event)
	# 	print("---")


ID: D
Type: initial_situation
Text: She lived quietly by the river, respected by her neighbors and unaware that her peaceful routine was about to end.
---
Thinking: ["The text describes a setup for an impending change in the protagonist's life, which is likely to be the main action of the story.", "The passage sets up a situation where the protagonist's life is about to change, indicating the beginning of a conflict or event.", "The text describes a setup for an impending change in the protagonist's life, introducing a contrast between her current peaceful state and the upcoming disruption.", "The narrative event described in this fragment is setting up a potential conflict or change in the protagonist's life, which suggests it's not yet a resolution. The focus on the protagonist's 'peaceful routine' being disrupted implies an action (move) is about to occur.", "The given fragment sets up a scene where the protagonist's life is about to change, introducing conflict or tension. This is 